In [1]:
import wrapper
import h5pyd

The wrapper function contains how user inputs are handled to provide an interpretation of the wealth of data available for clean energy technology implementation. 

The function defaults to use cases for the government (parameter `residential = False`). Given a h5pyd file (to interface with the API), a city, state, an amount of land available for technology installation, and a goal percentage (default to 100), the wrapper function outputs an interactive Altair chart. 

First, it interprests the city and state inputs to grab the location and relevant indices in the API h5pyd file (see location_hanlding_tests.ipynb). Then, it calculates the estimated energy use per capita in the state and multiplies that by the population of the city to get an estimate of total energy usage. Finally, it calculates how much energy the government wants covered by green technology. 

Using the wind and solar handling functions, we find how much energy your average solar panel and average wind turbine would output in the given location.

In theory, a government body could implement any combination of solar panels and wind turbines depending on their needs. However, the number of panels and turbines is restricted by the land available that the user indicated. That limitation can be represented as the number of turbines divided by the number of panels (for our purposes). 

Then, given the number of panels and turbines possible (in solar-only or wind-only investment scenarios), we form a numpy `meshgrid` of those to create an array of arrays containing every possibility of the implementation. For each combination, we calculate the estimated energy output, the cost (in millions of dollars), and the percent of their energy goal they can reach with that combination of wind and solar energy. 

Using Altair's delcarative visualization techniques, the function creates a scatter plot of number of turbines vs. number of 100,000 solar panels. The color of the plot represents the estimated energy output in MWh. If you hover over any point, it gives you the exact number of turbines and panels for that combination, the cost, and the percent of their energy goal it covers.

In [2]:
wtk = h5pyd.File('/nrel/wtk-us.h5', 'r')

Government of Salem, OR with 20 $km^2$ of land available for development.

In [3]:
chart = wrapper.wrapper(wtk, 'Salem', 'OR', land_available = 20, goal = 80)
chart

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [4]:
type(chart)

altair.vegalite.v3.api.LayerChart

For residential use cases, the wrapper function requires the h5pyd file, city, state, a energy conversion goal, and a household size. Land available is ignored (and therefore in the example below is an absurd 100,000 $km^2$ value). The residential parameter is set to True. The user may opt to input their monthly energy bill. If they do, we use that to calculate their yearly energy usage and then their energy goal. As laws vary from state to state regarding residential wind energy use, the wrapper function recommends only solar implementation for users.

To cover 100% (or otherwise high percentages) of energy use with green technology, often astronomical numbers of solar panels were required. This did not make sense for the average user's rooftop or their budget (as  >300 solar panels often cost >$50,000). At that point, the wrapper function output the energy output expected from some calculated number of solar panels and the cost of those solar panels. 

In addition to this, to make the output more usable and reasonable, we decided to show what 25 solar panels can do (as ~25 solar panels can fit on the average roof). In addition to the other values, the wrapper function outputs the energy output for 25 panels in that given location, the percent of total energy use that those 25 panels saves the user, and the cost of 25 panels. As cost is dependent on solar resources (see cost_handling_tests.ipynb), we hoped this would be a more useful output for a residential user.

Residential household of 8 from Salem, OR wanting to convert to 100% renewable energy.

In [6]:
result = wrapper.wrapper(wtk, 'Salem', 'OR', 100000, goal = 100, residential = True, household_size = 8)
print('Expected energy output: ' + str(result[0]) + ' kWh')
print('Recommended number of solar panels: ' + str(result[1]))
print('Estimated cost: ' + str(result[2]))
print('Expected energy output of 25 panels: ' + str(result[3]))
print('Expected energy saved from installing 25: ' + str(result[4]) + '%')
print('Cost of 25 solar panels: ' + str(result[5]))

Expected energy output: 149129.03 kWh
Recommended number of solar panels: 307.0
Estimated cost: 53454.93
Expected energy output of 25 panels: 12144.06
Expected energy saved from installing 25: 8.13%
Cost of 25 solar panels: 4353.01


In [7]:
result

[149129.03, 307.0, 53454.93, 12144.06, 8.13, 4353.01]